<a href="https://colab.research.google.com/github/diego-lazaro/deep_learning_challenge/blob/main/AlphabetSoupCharity_Optimization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID column, 'EIN'.
application_df = application_df.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df["APPLICATION_TYPE"].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts[counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_above_one = classification_counts[classification_counts > 1]
classification_counts_above_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts_above_one[classification_counts_above_one < 100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      643
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1900        1
C2600        1
C1248        1
C6100        1
C1820        1
C1580        1
C2570        1
C1732        1
C3700        1
C2500        1
C1570        1
C1283        1
C2380        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 48)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 16
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                5920      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 16)                496       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 8,863
Trainable params: 8,863
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5689 - accuracy: 0.7200
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7343
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5445 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5446 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5440 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5433 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7265 - 482ms/epoch - 2ms/step
Loss: 0.5576563477516174, Accuracy: 0.7265306115150452


In [17]:
# Optimize neural net #1
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 14)                1036      
                                                                 
 dense_5 (Dense)             (None, 20)                300       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,357
Trainable params: 1,357
Non-trainable params: 0
_________________________________________________________________


In [18]:

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the optimized model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5953 - accuracy: 0.6981
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5591 - accuracy: 0.7260
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7285
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7311
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7319
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7324
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7320
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7326
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accuracy: 0.7334

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5483 - accuracy: 0.7306 - 688ms/epoch - 3ms/step
Loss: 0.5482823848724365, Accuracy: 0.7306122183799744


In [21]:
# Train the optimized model with 150 epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5401 - accuracy: 0.7357
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7360
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5404 - accuracy: 0.7366
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5402 - accuracy: 0.7368
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5400 - accuracy: 0.7363
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5397 - accuracy: 0.7369
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5399 - accuracy: 0.7366
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5396 - accuracy: 0.7365
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5398 - accuracy: 0.7371
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5398 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5505 - accuracy: 0.7276 - 321ms/epoch - 1ms/step
Loss: 0.5505267381668091, Accuracy: 0.727580189704895


In [23]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_optimizer_2.h5')
files.download('/content/AlphabetSoupCharity_optimizer_2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# Fin